# Airplane Cargo Analysist

**Metadata:**

https://data.sfgov.org/Transportation/Air-Traffic-Cargo-Statistics/u397-j8nr/about_data [san francisco air cargo data]

**Case:**
* Evaluate the performance of different airlines based on cargo metrics, identifying top performers and areas for improvement.
* Analyze patterns in different types of cargo activities to understand trends in the industry, such as the prevalence of express shipments or bulk cargo.
* Examine cargo operations over time to identify seasonality, peak periods, or long-term trends in air cargo demand.
* Benchmark the performance of specific airlines, regions, or aircraft types against industry averages to assess competitiveness
* Utilize historical data to build predictive models for future air cargo trends, helping stakeholders make informed decisions.

## Data Wrangling

In [1]:
#import necessary library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from fuzzywuzzy import fuzz,process

#set all things that needed
pd.set_option('display.max_columns', 100)
plt.style.use('dark_background')

df = pd.read_csv('/kaggle/input/air-traffic-cargo-statistic/Air_Traffic_Cargo_Statistics.csv')

In [2]:
df.shape

(54043, 15)

In [3]:
df.head()

,Activity Period,Activity Period Start Date,Operating Airline,Operating Airline IATA Code,Published Airline,Published Airline IATA Code,GEO Summary,GEO Region,Activity Type Code,Cargo Type Code,Cargo Aircraft Type,Cargo Weight LBS,Cargo Metric TONS,data_as_of,data_loaded_at
0,199907,1999/07/01,ATA Airlines,TZ,ATA Airlines,TZ,Domestic,US,Deplaned,Cargo,Passenger,349048,158.328173,2023/12/20 02:00:06 PM,2023/12/20 04:48:03 PM
1,199907,1999/07/01,ATA Airlines,TZ,ATA Airlines,TZ,Domestic,US,Deplaned,Mail,Passenger,122003,55.340561,2023/12/20 02:00:06 PM,2023/12/20 04:48:03 PM
2,199907,1999/07/01,ATA Airlines,TZ,ATA Airlines,TZ,Domestic,US,Enplaned,Cargo,Passenger,514450,233.354520,2023/12/20 02:00:06 PM,2023/12/20 04:48:03 PM
3,199907,1999/07/01,ATA Airlines,TZ,ATA Airlines,TZ,Domestic,US,Enplaned,Mail,Passenger,89776,40.722394,2023/12/20 02:00:06 PM,2023/12/20 04:48:03 PM
4,199907,1999/07/01,Aeroflot Russian International Airlines,NaN,Aeroflot Russian International Airlines,NaN,International,Europe,Deplaned,Cargo,Passenger,13086,5.935810,2023/12/20 02:00:06 PM,2023/12/20 04:48:03 PM


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54043 entries, 0 to 54042
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Activity Period              54043 non-null  int64  
 1   Activity Period Start Date   54043 non-null  object 
 2   Operating Airline            54043 non-null  object 
 3   Operating Airline IATA Code  53465 non-null  object 
 4   Published Airline            54043 non-null  object 
 5   Published Airline IATA Code  53465 non-null  object 
 6   GEO Summary                  54043 non-null  object 
 7   GEO Region                   54043 non-null  object 
 8   Activity Type Code           54043 non-null  object 
 9   Cargo Type Code              54043 non-null  object 
 10  Cargo Aircraft Type          54043 non-null  object 
 11  Cargo Weight LBS             54043 non-null  int64  
 12  Cargo Metric TONS            54043 non-null  float64
 13  data_as_of      

In [5]:
#select all columns that we need
df = df[[
#     'Activity Period',
    'Activity Period Start Date',
    'Operating Airline',
#     'Operating Airline IATA Code',
    'Published Airline',
#     'Published Airline IATA Code',
    'GEO Summary',
    'GEO Region',
    'Activity Type Code',
    'Cargo Type Code',
    'Cargo Aircraft Type',
    'Cargo Weight LBS',
    'Cargo Metric TONS',
#     'data_as_of',
#     'data_loaded_at'
]]

In [6]:
#change all columns name
df = df.rename(columns={
    'Activity Period Start Date' : 'activity_period_start_date',
    'Operating Airline' : 'operating_airline',
    'Published Airline' : 'published_airline',
    'GEO Summary' : 'geo_summary',
    'GEO Region' : 'geo_region',
    'Activity Type Code' : 'activity_type_code',
    'Cargo Type Code' : 'cargo_type_code',
    'Cargo Aircraft Type' : 'cargo_aircraft_type',
    'Cargo Weight LBS' : 'cargo_weight_lbs',
    'Cargo Metric TONS' : 'cargo_metric_tons'
})

df

,activity_period_start_date,operating_airline,published_airline,geo_summary,geo_region,activity_type_code,cargo_type_code,cargo_aircraft_type,cargo_weight_lbs,cargo_metric_tons
0,1999/07/01,ATA Airlines,ATA Airlines,Domestic,US,Deplaned,Cargo,Passenger,349048,158.328173
1,1999/07/01,ATA Airlines,ATA Airlines,Domestic,US,Deplaned,Mail,Passenger,122003,55.340561
2,1999/07/01,ATA Airlines,ATA Airlines,Domestic,US,Enplaned,Cargo,Passenger,514450,233.354520
3,1999/07/01,ATA Airlines,ATA Airlines,Domestic,US,Enplaned,Mail,Passenger,89776,40.722394
4,1999/07/01,Aeroflot Russian International Airlines,Aeroflot Russian International Airlines,International,Europe,Deplaned,Cargo,Passenger,13086,5.935810
...,...,...,...,...,...,...,...,...,...,...
54038,2023/10/01,Vietnam Airlines JSC,Vietnam Airlines JSC,International,Asia,Deplaned,Cargo,Passenger,286532,129.970915
54039,2023/10/01,Vietnam Airlines JSC,Vietnam Airlines JSC,International,Asia,Enplaned,Cargo,Passenger,67673,30.696473
54040,2023/10/01,Virgin Atlantic,Virgin Atlantic,International,Europe,Deplaned,Cargo,Passenger,212137,96.225343
54041,2023/10/01,Virgin Atlantic,Virgin Atlantic,International,Europe,Enplaned,Cargo,Passenger,196029,88.918754


In [7]:
pd.set_option('display.max_rows',200)
df['operating_airline'].value_counts().sort_index()

operating_airline
21 Air, LLC                                  16
ABX Air                                     334
ASTAR USA, LLC                               16
ATA Airlines                                324
Active Aero Charter                           2
Aer Lingus, Ltd.                            255
Aeroflot Russian International Airlines     112
Aeromexico                                    9
Air Berlin                                   60
Air Canada                                 1596
Air Canada Jazz                               2
Air Cargo Carriers                          140
Air China                                  1095
Air France                                 1034
Air India Limited                           181
Air Italy S.P.A                              14
Air New Zealand                             847
Air Pacific Limited dba Fiji Airways        122
Air Transport International, Inc.            88
Air Transport International, LLC.             6
Alaska Airlines       

In [8]:
#menggabungkan maskapai airline yang sama
df['operating_airline'] = df['operating_airline'].replace('United Airlines - Pre 07/01/2013', 'United Airlines')
df['published_airline'] = df['published_airline'].replace('United Airlines - Pre 07/01/2013', 'United Airlines')

In [9]:
#mengubah datetime nya ke dtype seharusnya
df['activity_period_start_date'] = pd.to_datetime(df['activity_period_start_date'])

In [10]:
# want to know what years the airline start operation
# df.query('operating_airline == "Atlas Air, Inc"').sort_values(by='activity_period_start_date', ascending=False)

In [11]:
df.describe()

,activity_period_start_date,cargo_weight_lbs,cargo_metric_tons
count,54043,5.404300e+04,54043.000000
mean,2011-04-10 05:33:47.252372992,5.194877e+05,235.639615
min,1999-07-01 00:00:00,1.000000e+00,0.000454
25%,2005-02-01 00:00:00,1.961600e+04,8.897818
50%,2011-02-01 00:00:00,1.443320e+05,65.468995
75%,2017-07-01 00:00:00,5.918965e+05,268.484252
max,2023-10-01 00:00:00,2.381234e+07,10801.278331
std,NaN,1.015188e+06,460.489304


In [12]:
df.describe(include='object')

,operating_airline,published_airline,geo_summary,geo_region,activity_type_code,cargo_type_code,cargo_aircraft_type
count,54043,54043,54043,54043,54043,54043,54043
unique,129,118,2,9,2,3,3
top,United Airlines,United Airlines,International,Asia,Deplaned,Cargo,Passenger
freq,9251,9272,37735,19012,27630,29954,44391
